# Modelling metabolism in SORGHUM

### Generating SORGHUM core model

In [1]:
import pandas as pd
from cobra import io
from cobra.core import Metabolite, Reaction
from cobra import flux_analysis
from Functions import *
from IPython import display
import logging
logging.basicConfig()
logger = logging.getLogger('logger')

#import sbml file
fname = "./../plantcoremetabolism-model/PlantCoreMetabolism_v2_1_0.xml"
model = io.read_sbml_model(fname)
model.solver='glpk'
display.clear_output()



#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("AraCore_Biomass_tx")
rxn.remove_from_model()

#remove charged state of PGPs since they are minor (0.03)
model=removeSpecificMetChargedState(model,["aL_1_PHOSPHATIDYL_GLYCEROL_P_p","aL_1_PHOSPHATIDYL_GLYCEROL_P_m"])

In [2]:
import pandas as pd

df = pd.read_csv("Data/biomass_sorghum.csv")

FA=["PALMITATE_p","CPD_9245_p","CPD_17412_p","CPD_17291_p","STEARIC_ACID_p","OLEATE_CPD_p",
    "Octadecadienoate_p","LINOLENIC_ACID_p","ARACHIDIC_ACID_p","CPD_16709_p","DOCOSANOATE_p"]


In [3]:
biomass = df.set_index("Unnamed: 0")

### Use biomass composition to generate biomass equations

### Run stem

In [4]:
from cobra import flux_analysis
stem_model = model.copy()

k = "stem"

stem_model = updateFAcomposition(stem_model,k,biomass)

rxn = Reaction("Biomass_stem_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    else:
        rxn.add_metabolites({stem_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
stem_model.add_reaction(rxn)

stem_model = generateStemModel(stem_model)

for rxn in stem_model.reactions:
    if rxn.id=="SUCROSE_ph_exchange":
        rxn.lower_bound = 1
        rxn.upper_bound = 1
    elif "ph_exchange" in rxn.id:
        rxn.lower_bound = 0
        rxn.upper_bound = 0

stem_model.reactions.GLC_tx.upper_bound = 0
stem_model.reactions.GLC_tx.lower_bound = 0
stem_model.reactions.Sucrose_tx.upper_bound = 0
stem_model.reactions.Sucrose_tx.lower_bound = 0
stem_model.reactions.Photon_tx.upper_bound = 0
stem_model.reactions.Photon_tx.lower_bound = 0
stem_model.reactions.Biomass_stem_tx.objective_coefficient=1

flux_analysis.parsimonious.pfba(stem_model)
print("Biomass flux ="+str(stem_model.reactions.Biomass_stem_tx.flux))

0.0
Biomass flux =0.3965874837730572


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 40.42553191489362 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 77.72340425531915 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 13.829787234042554 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.127659574468085 is not an integer (in formula C40.42553191489362H77.72340425531915O13.829787234042554P2.127659574468085)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.9999999999999996 is not an integer (in form

In [5]:
stem_model.reactions.query("Biomass_tx")

[<Reaction Biomass_tx at 0x1f7df4804c0>]

In [6]:
stem_model.reactions.Biomass_stem_tx.reaction

'0.728315974933178 COUMARATE_c + 1.7764240905725708 Cellulose_b + 0.38776720411991217 FRU_c + 0.4602470553572778 GLC_c + 0.8994069662814456 Xylan_b + 0.1068123162394869 sSUCROSE_b --> '

### Run root

In [7]:
from cobra import flux_analysis
root_model = model.copy()

k = "root"
root_model = updateFAcomposition(root_model,k,biomass)

rxn = Reaction("Biomass_root_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({root_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
root_model.add_reaction(rxn)

root_model = generateRootModel(root_model)

root_model.reactions.GLC_tx.upper_bound = 0
root_model.reactions.GLC_tx.lower_bound = 0
root_model.reactions.Sucrose_tx.upper_bound = 0
root_model.reactions.Sucrose_tx.lower_bound = 0
root_model.reactions.Photon_tx.upper_bound = 0
root_model.reactions.Photon_tx.lower_bound = 0


root_model.reactions.NH4_tx.upper_bound = 0
root_model.reactions.NH4_tx.lower_bound = 0
 
    
for rxn in root_model.reactions.query("_phloem_"):
    if "SUCROSE" not in rxn.id :
        rxn.lower_bound = 0
        rxn.upper_bound = 0

for rxn in root_model.reactions:
    rxn.objective_coefficient=0
root_model.reactions.Biomass_root_tx.objective_coefficient=1
flux_analysis.parsimonious.pfba(root_model)
print("Biomass flux = "+str(root_model.reactions.Biomass_root_tx.flux))

0.0
Biomass flux = 14.783982435687493


In [8]:
stem_model.reactions.Biomass_stem_tx.reaction

'0.728315974933178 COUMARATE_c + 1.7764240905725708 Cellulose_b + 0.38776720411991217 FRU_c + 0.4602470553572778 GLC_c + 0.8994069662814456 Xylan_b + 0.1068123162394869 sSUCROSE_b --> '

### Run seed

In [9]:
from cobra import flux_analysis
seed_model = model.copy()

seed_model = generateSeedModel(seed_model)

k = "seed"
seed_model = updateFAcomposition(seed_model,k,biomass)


for rxn in seed_model.reactions:
    if rxn.id=="SUCROSE_ph_exchange":
        rxn.lower_bound = 1
        rxn.upper_bound = 1
    elif "ph_exchange" in rxn.id:
        rxn.lower_bound = 0
        rxn.upper_bound = 0



rxn = Reaction("Biomass_seed_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({seed_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})
rxn.lower_bound = 0
rxn.upper_bound = 1000
seed_model.add_reaction(rxn)

seed_model.reactions.GLC_tx.upper_bound = 0
seed_model.reactions.GLC_tx.lower_bound = 0
seed_model.reactions.Sucrose_tx.upper_bound = 0
seed_model.reactions.Sucrose_tx.lower_bound = 0
seed_model.reactions.Photon_tx.upper_bound = 0
seed_model.reactions.Photon_tx.lower_bound = 0
seed_model.reactions.Biomass_seed_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(seed_model)
print("Biomass flux ="+str(seed_model.reactions.Biomass_seed_tx.flux))

    
    

0.0
Biomass flux =0.7498501513695344


In [10]:
seed_model.reactions.Biomass_seed_tx.reaction

'0.0290287408345124 ARG_c + 0.04964753924729917 COUMARATE_c + 0.009526544142541593 CYS_c + 0.08847893279975096 FRU_c + 0.09180938196410796 GLC_c + 0.05121597854438592 GLY_c + 0.0185023834727556 HIS_c + 0.043587502664421914 ILE_c + 0.14186704529664612 LEU_c + 0.019181547669595882 LYS_c + 0.013842921277520416 MET_c + 0.04418703765877103 PHE_c + 0.0980210478906567 PRO_c + 0.03573277660229751 THR_c + 0.0066399337567809545 TRP_c + 0.03001851230031804 TYR_c + 0.060842258637074514 VAL_c + 0.14252283395878335 sALA_b + 0.5513719420874239 sASP_b + 1.4793643163548391 sGLU_b + 0.05833794751890824 sSER_b + 0.012094778187610927 sSUCROSE_b --> '

### Generate diel leaf model



In [11]:

def generateMultiphaseModel(model,tags):
    models=dict()
    for i in range(0,len(tags)):
        temp=model.copy()
        for met in temp.metabolites:
            met.id = met.id+str(i+1)
            met.compartment = met.compartment+str(i+1)
            met.name = met.name+" "+tags[i]
        for rxn in temp.reactions:
            rxn.id = rxn.id+str(i+1)
            rxn.name = rxn.name+" "+tags[i]
        if i==0:
            modelOut = temp.copy()
        else:
            modelOut = modelOut+temp
            for met in temp.metabolites:
                if met.id not in modelOut.metabolites:
                    met2 = met.copy()
                    #print(met2.id)
                    modelOut.add_metabolites(met2)
        for comp in temp.compartments:
            modelOut.compartments={str(comp):model.compartments[comp[0:len(comp)-1]]+" "+tags[i]}
    return modelOut


def addLinkers(model,baseMets,from_phase,to_phase,reversible=True):
    from cobra.core import Reaction
    for met in baseMets.keys():
        baseMet = met
    rxn = Reaction(baseMet+"_linker"+from_phase+to_phase)
    rxn.name = rxn.id
    baseMets = baseMets[baseMet]
    for baseMetID in baseMets.keys():
        met1 = model.metabolites.get_by_id(baseMetID+from_phase)
        met2 = model.metabolites.get_by_id(baseMetID+to_phase)
        rxn.add_metabolites({met1:-1*baseMets[baseMetID],met2:1*baseMets[baseMetID]})
    rxn.upper_bound = 1000
    if reversible:
        rxn.lower_bound = -1000
    else:
        rxn.lower_bound = 0
    model.add_reaction(rxn)
    return model
    

def identifyFractionalMets(model):
    import re
    fractionMets=dict()
    for rxn in model.reactions:
        for met in rxn.metabolites.keys():
            prefix=""
            a=re.search("^a{1,3}",met.id)
            anion=""
            if a:
                anion=a.group(0)
                prefix=anion
            b=re.search("^b{1,3}",met.id)
            basic=""
            if b:
                basic=b.group(0)
                prefix=basic
            if (not prefix == ""):
                mainMet=model.metabolites.get_by_id(met.id[len(prefix):])
                fractionMets[met.id]=mainMet.id
                #fractionMets[met.id]=prefix
    return fractionMets


# def mergeCompartments(model,comp1,comp2):
#     temp = model.copy()
#     for met in model.metabolites:
#         if met.compartment==comp2:
#             M_orig = met.id.replace(comp2,comp1)
#             if M_orig in model.metabolites:
#                 M_orig = model.metabolites.get_by_id(M_orig)
#                 for rxn in M_orig.reactions:
#                     M2 = model.metabolites.get_by_id(M_orig.id)
#                     coeff = rxn.metabolites[M_orig]
#                     rxn.add_metabolites({M_orig:-1*coeff,M2:coeff})
                    
def addUpperAndLowerBoundsFromFile(model,filename):
    import pandas as pd
    df = pd.read_csv(filename,sep="\t")
    
    for rxn in df["reaction"]:
        rxn = model.reactions.get_by_id(str(rxn))
        rxn.lower_bound = float(df[df["reaction"]==rxn.id]["lower-bound"])
        rxn.upper_bound = float(df[df["reaction"]==rxn.id]["upper-bound"])
    
    return model


def generateC4leafModel(model,constraintsFile):
    tags = ["mesophyll day","mesophyll night","bundlesheath day","bundlesheath night"]
    C4_model = generateMultiphaseModel(model,tags)

    mets1 = ["STARCH_p","MAL_v","FUM_v","CIT_v","NITRATE_v","SUCROSE_v","GLC_v","FRU_v"]
    for met in mets1:
        if met == "MAL_v":
            mets = {met:{"MAL_v":0.7,"aMAL_v":0.3}}
        elif met == "FUM_v":
            mets = {met:{"FUM_v":0.92,"aFUM_v":0.08}}
        elif met == "CIT_v":
            mets = {met:{"CIT_v":0.5,"aCIT_v":0.5}}
        else:
            mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","2")
        C4_model = addLinkers(C4_model,mets,"3","4")

    mets2 = ["ASN_v","GLT_v","L_ALPHA_ALANINE_v","LEU_v","bHIS_v","MET_v","LYS_v","THR_v","VAL_v","PHE_v",
            "4_AMINO_BUTYRATE_v","SER_v","ARG_v","GLN_v","GLY_v","L_ASPARTATE_v","ILE_v","PRO_v","CYS_v",
            "TRP_v","TYR_v"]
    for met in mets2:
        mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","2",reversible=False)
        C4_model = addLinkers(C4_model,mets,"3","4",reversible=False)


    mets3 = ["MAL_c","PYRUVATE_c","ASN_c","GLT_c","L_ALPHA_ALANINE_c","LEU_c",
            "MET_c","LYS_c","HIS_c","THR_c","VAL_c","PHE_c","4_AMINO_BUTYRATE_c","SER_c","ARG_c","GLN_c",
            "GLY_c","L_ASPARTATE_c","ILE_c","PRO_c","CYS_c","TRP_c","TYR_c","G3P_c","GAP_c","Pi_c",
            "PHOSPHO_ENOL_PYRUVATE_c"] #"SUCROSE_c","MALTOSE_p",
    for met in mets3:
        if met=="Pi_c":
            mets = {met:{"Pi_c":0.7,"aPi_c":0.3}}
        else:
            mets = {met:{met:1}}
        C4_model = addLinkers(C4_model,mets,"1","3")
        C4_model = addLinkers(C4_model,mets,"2","4")


    C4_model = addUpperAndLowerBoundsFromFile(C4_model,constraintsFile)   

    #Nitrate ratio day:night
    met = Metabolite("Nitrate_pseudo")
    met.name = "Nitrate_pseudo"
    met.compartment = "f"
    met.charge = 0
    for i in (1,3):
        C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : 2})
    for i in (2,4):
        C4_model.reactions.get_by_id(str("Nitrate_tx" + str(i))).add_metabolites({met : -3})

    #Rubisco oxygenase
    for i in range(1,5):
        met = Metabolite("Rubisco_pseudo" + str(i))
        met.name = "Rubisco_pseudo" + str(i)
        met.compartment = "f"
        met.charge = 0
        C4_model.reactions.get_by_id(str("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p" + str(i))).add_metabolites({met : 1})
        C4_model.reactions.get_by_id(str("RXN_961_p" + str(i))).add_metabolites({met : -20})


    #Maintainence costs
    for i in range(1,5):
        C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).upper_bound = C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).lower_bound = 19.5
        met = Metabolite(str("Maint_pseudo" + str(i)))
        met.name = str("Maint_pseudo" + str(i))
        met.compartment = "f"
        met.charge = 0
        C4_model.reactions.get_by_id(str("ATPase_tx" + str(i))).add_metabolites({met : 1})
        C4_model.reactions.get_by_id(str("NADPHoxc_tx" + str(i))).add_metabolites({met : -3})
        C4_model.reactions.get_by_id(str("NADPHoxm_tx" + str(i))).add_metabolites({met : -3})
        C4_model.reactions.get_by_id(str("NADPHoxp_tx" + str(i))).add_metabolites({met : -3})



    for i in range(3,5):
        for met in model.metabolites:
            if "CARBON_DIOXIDE" in met.id or "OXYGEN_MOLECULE" in met.id or "Photon" in met.id:
                continue
            if met.compartment == "e"+str(i):
                rxn = Reaction(met.id+"_e"+str(i-2)+str(i))
                rxn.add_metabolites({met:1,C4_model.metabolites.get_by_id(met.id.replace(met.compartment,"e"+str(i-2))):-1})




    met1 = Metabolite("X_Phloem_contribution_day",name="Phloem output during the day",compartment="b1")
    C4_model.reactions.get_by_id("Phloem_output_tx1").remove_from_model()
    C4_model.reactions.get_by_id("Phloem_output_tx3").add_metabolites({met1:1})
    met2 = Metabolite("X_Phloem_contribution_night",name="Phloem output during at night",compartment="b1")
    C4_model.reactions.get_by_id("Phloem_output_tx2").remove_from_model()
    C4_model.reactions.get_by_id("Phloem_output_tx4").add_metabolites({met2:1})

    rxn = Reaction("diel_biomass")
    rxn.add_metabolites({met1:-0.75,met2:-0.25})
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    C4_model.add_reaction(rxn)
    rxn.objective_coefficient=1
    return C4_model



In [13]:
from cobra import flux_analysis
leaf_model = model.copy()

#change Biomass_tx to Protein_biomass
rxn = leaf_model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = leaf_model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
met.compartment = "b"
rxn.add_metabolites({met:1})

k = "leaf"
leaf_model = updateFAcomposition(leaf_model,k,biomass)

rxn = Reaction("Biomass_leaf_tx")
for met in df["Unnamed: 0"]:
    if met in FA or float(df[df["Unnamed: 0"]==met][k])==0:
        continue
    rxn.add_metabolites({leaf_model.metabolites.get_by_id(met):-1*float(df[df["Unnamed: 0"]==met][k])})

rxn.lower_bound = 0
rxn.upper_bound = 1000
leaf_model.add_reaction(rxn)

leaf_model.reactions.GLC_tx.upper_bound = 0
leaf_model.reactions.GLC_tx.lower_bound = 0
leaf_model.reactions.Sucrose_tx.upper_bound = 0
leaf_model.reactions.Sucrose_tx.lower_bound = 0
leaf_model.reactions.Photon_tx.upper_bound = 0
leaf_model.reactions.Photon_tx.lower_bound = 0
leaf_model.reactions.Biomass_leaf_tx.objective_coefficient=1
sol=flux_analysis.parsimonious.pfba(leaf_model)
print("Biomass flux ="+str(leaf_model.reactions.Biomass_leaf_tx.flux))

84.3439911797135
Biomass flux =0.0


C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 17.138823529411766 is not an integer (in formula C17.138823529411766H31.157908496732034O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 31.157908496732034 is not an integer (in formula C17.138823529411766H31.157908496732034O2)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 38.138823529411766 is not an integer (in formula C38.138823529411766H66.15790849673203N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 66.15790849673203 is not an integer (in formula C38.138823529411766H66.15790849673203N7.0O17.0P2.9999999999999996S1.0)
  warn(
C:\ProgramData\Anaconda3\lib\site-packages\cobra\core\metabolite.py:103: UserWarning: 2.9999999999999996 is not an integer (in formula C38.138823529411766H66.15790849673203N7.0O17.0P2.9999999999999996S1.

In [14]:
leaf_model_source = generateC4leafModel(leaf_model,"Sorghum_Leaf_Constraints.txt")

leaf_model_source.reactions.Photon_tx1.upper_bound = 600
leaf_model_source.reactions.Photon_tx3.upper_bound = 400

sol = flux_analysis.parsimonious.pfba(leaf_model_source)
print("Phloem output = ")
print(leaf_model_source.reactions.diel_biomass.flux)

Phloem output = 
5.054115122058


In [15]:
for i in range(1,5):
    for rxn in ["CO2_tx","O2_tx","PSII_RXN_p","RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p",
                "PEPCARBOXYKIN_RXN_c","1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m","MALIC_NADP_RXN_p",
                "MALIC_NADP_RXN_c"]:
        print(rxn+str(i)+" flux = "+str(sol.fluxes[rxn+str(i)]))
    print("------")

CO2_tx1 flux = 51.187972893714125
O2_tx1 flux = -71.75000000000001
PSII_RXN_p1 flux = 75.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p1 flux = 0.0
PEPCARBOXYKIN_RXN_c1 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m1 flux = 0.0
MALIC_NADP_RXN_p1 flux = 0.0
MALIC_NADP_RXN_c1 flux = 0.0
------
CO2_tx2 flux = -0.2541591154157307
O2_tx2 flux = 4.317786044895263
PSII_RXN_p2 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p2 flux = 0.0
PEPCARBOXYKIN_RXN_c2 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m2 flux = 0.0
MALIC_NADP_RXN_p2 flux = 0.0
MALIC_NADP_RXN_c2 flux = 0.0
------
CO2_tx3 flux = 0.0
O2_tx3 flux = 8.059758871391294
PSII_RXN_p3 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p3 flux = 64.13011828521705
PEPCARBOXYKIN_RXN_c3 flux = 0.0
1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m3 flux = 0.0
MALIC_NADP_RXN_p3 flux = 62.28615505423826
MALIC_NADP_RXN_c3 flux = 0.0
------
CO2_tx4 flux = 0.0
O2_tx4 flux = 7.376547970763605
PSII_RXN_p4 flux = 0.0
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p4 flux = 0.0


In [16]:
growing_leaf_model = leaf_model_source.copy()

growing_leaf_model.reactions.Biomass_leaf_tx1.upper_bound=1000
growing_leaf_model.reactions.Biomass_leaf_tx2.upper_bound=1000
growing_leaf_model.reactions.Biomass_leaf_tx3.upper_bound=1000
growing_leaf_model.reactions.Biomass_leaf_tx4.upper_bound=1000

met1=Metabolite("Leaf_biomass1",compartment="c1")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx1
rxn.add_metabolites({met1:1})

met2=Metabolite("Leaf_biomass2",compartment="c2")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx2
rxn.add_metabolites({met2:1})

met3=Metabolite("Leaf_biomass3",compartment="c3")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx3
rxn.add_metabolites({met3:1})

met4=Metabolite("Leaf_biomass4",compartment="c4")
rxn = growing_leaf_model.reactions.Biomass_leaf_tx4
rxn.add_metabolites({met4:1})

growing_leaf_model.reactions.diel_biomass.add_metabolites({growing_leaf_model.metabolites.X_Phloem_contribution_day:3,
                                                           growing_leaf_model.metabolites.X_Phloem_contribution_night:1,
                                                           met1:-0.75, met2:-0.25,met3:-0.75, met4:-0.25})


from cobra.flux_analysis import pfba
sol = pfba(growing_leaf_model)
print("Biomass accumulation rate rate ="+str(sol.fluxes["diel_biomass"]))

Biomass accumulation rate rate =0.0


In [18]:
io.write_sbml_model(root_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\sorghum_root_model.xml")

In [19]:
io.write_sbml_model(stem_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\sorghum_stem_model.xml")

In [21]:
for met in leaf_model_source.metabolites:
    if type(met.charge)!=int:
        print(met.id)
        print(met.charge)
        met.charge=0

Protein_b1
-0.09299016640000002
Protein_b2
-0.09299016640000002
Protein_b3
-0.09299016640000002
Protein_b4
-0.09299016640000002
X_Phloem_contribution_day
None
X_Phloem_contribution_night
None


In [22]:
io.write_sbml_model(leaf_model_source,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\sorghum_sourceleaf_model.xml")

In [23]:
for met in growing_leaf_model.metabolites:
    if type(met.charge)!=int:
        print(met.id)
        print(met.charge)
        met.charge=0

Protein_b1
-0.09299016640000002
Protein_b2
-0.09299016640000002
Protein_b3
-0.09299016640000002
Protein_b4
-0.09299016640000002
X_Phloem_contribution_day
None
X_Phloem_contribution_night
None
Leaf_biomass1
None
Leaf_biomass2
None
Leaf_biomass3
None
Leaf_biomass4
None


In [24]:
io.write_sbml_model(growing_leaf_model,"C:\\Users\\sanus\\Scripts\\git\\tissue-specific-FBA-models\\Models\\sorghum_sinkleaf_model.xml")